In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer

#### Read and clean benchmark data

In [ ]:
def clean_fn(df, col_name='text'):
    """Clean function used for preparing train/val data"""
    
    df.loc[:, col_name] = df[col_name].str.replace('https?://\S+|www\.\S+', ' social medium ')
    df.loc[:, col_name] = df[col_name].str.replace('\s+', ' ')  # remove more than 1 white space
    df.loc[:, col_name] = df[col_name].str.strip()
    
    return df

In [ ]:
benchmark_dir = Path('')

benchmark_data = pd.read_csv(benchmark_dir/'validation_data.csv')
benchmark_data.head()

In [ ]:
benchmark_data = clean_fn(benchmark_data.copy(), col_name='less_toxic')
benchmark_data = clean_fn(benchmark_data.copy(), col_name='more_toxic')

benchmark_data.head()

#### Encode data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

max_length = 128

In [ ]:
less_toxic_encoded, more_toxic_encoded = [], []

for idx in tqdm(benchmark_data.index.values, total=benchmark_data.shape[0]):
    lt_encoded = tokenizer.encode_plus(
        benchmark_data.loc[idx, 'less_toxic'],
        truncation=True,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length'
    )
    
    mt_encoded = tokenizer.encode_plus(
        benchmark_data.loc[idx, 'more_toxic'],
        truncation=True,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length'
    )
    
    less_toxic_encoded.append(lt_encoded)
    more_toxic_encoded.append(mt_encoded)

#### Path data threw model and get scores

In [ ]:
less_toxic_scores, more_toxic_scores = [], []

batch_size = 512
steps = len(less_toxic_encoded) // batch_size + 1

for idx in tqdm(range(steps)):
    
    minibatch = less_toxic_encoded[idx * batch_size:(idx + 1) * batch_size]
    
    lt_input_ids = [sample['input_ids'] for sample in minibatch]
    lt_attn_mask = [sample['attention_mask'] for sample in minibatch]
    
    lt_model_inp = np.stack([lt_input_ids, lt_attn_mask], axis=-1)
    
    lt_scores = model(lt_model_inp, training=False).numpy().squeeze()
    
    less_toxic_scores.extend(lt_scores)

for idx in tqdm(range(steps)):
    
    minibatch = more_toxic_encoded[idx * batch_size:(idx + 1) * batch_size]
    
    mt_input_ids = [sample['input_ids'] for sample in minibatch]
    mt_attn_mask = [sample['attention_mask'] for sample in minibatch]
    
    mt_model_inp = np.stack([mt_input_ids, mt_attn_mask], axis=-1)
    
    mt_scores = model(mt_model_inp, training=False).numpy().squeeze()
    
    more_toxic_scores.extend(mt_scores)

#### Get accuracy

In [ ]:
scores  = []
for score_less_toxic_comment, score_more_toxic_comment in zip(less_toxic_scores, more_toxic_scores):
    if score_less_toxic_comment < score_more_toxic_comment:
        scores.append(1)
    else:
        scores.append(0)

total_score = np.mean(scores)

In [ ]:
print(f'Accuracy: {total_score}')